In [0]:
# librairies
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window


In [0]:
# import des données
df = spark.read.json("s3://full-stack-bigdata-datasets/Big_Data/Project_Steam/steam_game_output.json")

In [0]:
# Afficher le schéma
df.printSchema()

# Afficher quelques lignes
display(df.limit(5))

# Afficher le nombre total de jeux
print("Nombre total de jeux :", df.count())

root
 |-- data: struct (nullable = true)
 |    |-- appid: long (nullable = true)
 |    |-- categories: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- ccu: long (nullable = true)
 |    |-- developer: string (nullable = true)
 |    |-- discount: string (nullable = true)
 |    |-- genre: string (nullable = true)
 |    |-- header_image: string (nullable = true)
 |    |-- initialprice: string (nullable = true)
 |    |-- languages: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- negative: long (nullable = true)
 |    |-- owners: string (nullable = true)
 |    |-- platforms: struct (nullable = true)
 |    |    |-- linux: boolean (nullable = true)
 |    |    |-- mac: boolean (nullable = true)
 |    |    |-- windows: boolean (nullable = true)
 |    |-- positive: long (nullable = true)
 |    |-- price: string (nullable = true)
 |    |-- publisher: string (nullable = true)
 |    |-- release_date: string (nullable = true)
 |    |-

data,id
"List(10, List(Multi-player, Valve Anti-Cheat enabled, Online PvP, Shared/Split Screen PvP, PvP), 13990, Valve, 0, Action, https://cdn.akamai.steamstatic.com/steam/apps/10/header.jpg?t=1666823513, 999, English, French, German, Italian, Spanish - Spain, Simplified Chinese, Traditional Chinese, Korean, Counter-Strike, 5199, 10,000,000 .. 20,000,000, List(true, true, true), 201215, 999, Valve, 2000/11/1, 0, Play the world's number 1 online action game. Engage in an incredibly realistic brand of terrorist warfare in this wildly popular team-based game. Ally with teammates to complete strategic missions. Take out enemy sites. Rescue hostages. Your role affects your team's success. Your team's success affects your role., List(266, 1191, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 5426, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 227, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 2784, null, null, null, null, null, null, null, null, null, null, null, null, 1607, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 4831, null, null, null, null, null, null, null, null, null, 1707, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 632, null, null, null, null, null, null, null, null, null, null, null, 3392, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 131, null, null, 769, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 881, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 289, null, null, null, 3353, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 614, null, null, null, null, null, null, 304, null, null, null, 1344, null, null, 1864, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 1192), game, )",10
"List(1000000, List(Single-player, Partial Controller Support, Steam Achievements, Steam Cloud), 0, IndigoBlue Game Studio, 0, Action, Adventure, Indie, https://cdn.akamai.steamstatic.com/steam/apps/1000000/header.jpg?t=1655723048, 999, English, Korean, Simplified Chinese, ASCENXION, 5, 0 .. 20,000, List(false, false, true), 27, 999, PsychoFlux Entertainment, 2021/05/14, 0, ASCENXION is a 2D shoot 'em up game where you explore the field to progress. Players must overcome puzzles, traps, elite units, boss fights, and other various obstacles while navigating the field. Grow stronger through rewards earned, 

Nombre total de jeux : 55691


## Which publisher hast released the most games on Steam ?

In [0]:
# nombre de jeux par éditeur classé par ordre decroissant
df.select('data.publisher') \
  .groupby('publisher') \
  .count() \
  .orderBy(F.col('count').desc()) \
  .show(5)


+--------------+-----+
|     publisher|count|
+--------------+-----+
|Big Fish Games|  422|
|        8floor|  202|
|          SEGA|  165|
|Strategy First|  151|
|   Square Enix|  141|
+--------------+-----+
only showing top 5 rows



## What are the best rated games?

A la vue des différentes colonnes, il y a plusieurs manière d'interpréter la question. Voici deux approches différentes :

In [0]:
# meilleur jeux selon ratio avis positifs/négatifs :
df.select('data.name', 'data.positive', 'data.negative')\
  .withColumn('ratio_positif', (F.col('positive') / (F.col('positive') + F.col('negative')) * 100))\
  .orderBy(F.col('ratio_positif').desc())\
  .show(10, False)

+---------------------+--------+--------+-------------+
|name                 |positive|negative|ratio_positif|
+---------------------+--------+--------+-------------+
|GUIDE                |2       |0       |100.0        |
|Rotund Rebound       |5       |0       |100.0        |
|Grab Lab             |9       |0       |100.0        |
|Next Stop 3          |17      |0       |100.0        |
|Cube Defender        |6       |0       |100.0        |
|Fist Of Heaven & Hell|7       |0       |100.0        |
|Diamonds             |4       |0       |100.0        |
|Twisting Mower       |1       |0       |100.0        |
|The Demon Crystal    |11      |0       |100.0        |
|Kill Tiger           |1       |0       |100.0        |
+---------------------+--------+--------+-------------+
only showing top 10 rows



Pas très parlant car un jeu avec 1 avis positif et 0 négatif peut être un jeu avec un ratio positif de 100%. La deuxième approche sera donc de classer les jeux par le nombre d'avis positifs

In [0]:
# Jeux classé par volume total d'avis positifs (popularité)
df.select('data.name', 'data.positive')\
  .orderBy(F.col('positive').desc())\
  .show(10, False)

+--------------------------------+--------+
|name                            |positive|
+--------------------------------+--------+
|Counter-Strike: Global Offensive|5943345 |
|Dota 2                          |1534895 |
|Grand Theft Auto V              |1229265 |
|PUBG: BATTLEGROUNDS             |1185361 |
|Terraria                        |1014711 |
|Tom Clancy's Rainbow Six Siege  |942910  |
|Garry's Mod                     |861240  |
|Team Fortress 2                 |846407  |
|Rust                            |732513  |
|Left 4 Dead 2                   |643836  |
+--------------------------------+--------+
only showing top 10 rows



## Are there years with more releases?

In [0]:
# Nombre de jeux sortis par années
df.select(F.substring('data.release_date', 1, 4).alias('year'))\
  .groupBy('year')\
  .count()\
  .orderBy(F.col('count').desc())\
  .show()

+----+-----+
|year|count|
+----+-----+
|2021| 8823|
|2020| 8305|
|2018| 7678|
|2022| 7455|
|2019| 6968|
|2017| 6017|
|2016| 4185|
|2015| 2576|
|2014| 1557|
|2013|  471|
|2012|  345|
|2009|  311|
|2010|  288|
|2011|  267|
|2008|  159|
|    |   99|
|2007|   98|
|2006|   61|
|2005|    6|
|2004|    6|
+----+-----+
only showing top 20 rows



Il y a une tendance générale à la hausse avec une forte croissance du nombre de sorties entre 2013 (471 jeux) et 2021 (8823 jeux)et une augmentation particulièrement marquée après 2014. 

A noter : 
- Il y a 99 jeux sans date de sortie (" ")
- Les données commencent en 2004 avec seulement 6 jeux

## How are the prizes distributed?

Commencons par regarder le prix moyen. les prix sont en cents donc je divise par 100 pour ramener en dollars.

In [0]:
# prix moyen des jeux
df.select(F.avg('data.price')/100)\
  .show()

+-----------------------+
|(avg(data.price) / 100)|
+-----------------------+
|      7.732849832109317|
+-----------------------+



Le prix moyen sur steam est de 7.73 dollars. Affinons la question afin de visualiser la répartition par prix.
A noter que précédement .avg arrive a convertir des string directement alors qu'avec des "opérations" il faut convertir en float. floor permet d'arrondir à l'entier inférieur

In [0]:
df.select(F.floor(F.col('data.price').cast('float')/100).alias('price_range_dollars'))\
  .groupBy('price_range_dollars')\
  .count()\
  .orderBy('price_range_dollars')\
  .show(30)

+-------------------+-----+
|price_range_dollars|count|
+-------------------+-----+
|                  0|13815|
|                  1| 4495|
|                  2| 3904|
|                  3| 2682|
|                  4| 6362|
|                  5| 1931|
|                  6| 1924|
|                  7| 1642|
|                  8|  782|
|                  9| 6171|
|                 10|  464|
|                 11|  841|
|                 12|  829|
|                 13|  292|
|                 14| 2885|
|                 15|  247|
|                 16|  192|
|                 17|  295|
|                 18|  144|
|                 19| 2833|
|                 20|   12|
|                 21|   11|
|                 22|   11|
|                 23|   11|
|                 24|  865|
|                 25|    5|
|                 26|    6|
|                 27|    2|
|                 28|    2|
|                 29|  869|
+-------------------+-----+
only showing top 30 rows



Il y a beaucoup trop de prix différents pour que cela soit lisible...Regroupons cela par catégorie de prix pour finir :

In [0]:
# cLassification des différents prix
df.select('data.price')\
  .withColumn('price_in_dollars', F.col('price').cast('float')/100)\
  .withColumn('price_category', 
    F.when(F.col('price_in_dollars') == 0, 'Free to play')
    .when(F.col('price_in_dollars') < 5, 'Moins de 5$')
    .when(F.col('price_in_dollars') < 10, '5$ - 9.99$')
    .when(F.col('price_in_dollars') < 15, '10$ - 14.99$')
    .when(F.col('price_in_dollars') < 20, '15$ - 19.99$')
    .when(F.col('price_in_dollars') < 30, '20$ - 29.99$')
    .when(F.col('price_in_dollars') < 50, '30$ - 49.99$')
    .otherwise('50$ et plus'))\
  .groupBy('price_category')\
  .count()\
  .orderBy('count', ascending=False)\
  .show(truncate=False)


+--------------+-----+
|price_category|count|
+--------------+-----+
|Moins de 5$   |23478|
|5$ - 9.99$    |12450|
|Free to play  |7780 |
|10$ - 14.99$  |5311 |
|15$ - 19.99$  |3711 |
|20$ - 29.99$  |1794 |
|30$ - 49.99$  |839  |
|50$ et plus   |328  |
+--------------+-----+



## Are there many games with a discount?

In [0]:
# Jeux  en promotions vs pas de promotion :
df.select('data.discount')\
  .withColumn('has_discount', 
    F.when(F.col('discount') != '0', 'Avec réduction')
    .otherwise('Sans réduction'))\
  .groupBy('has_discount')\
  .count()\
  .show()

+--------------+-----+
|  has_discount|count|
+--------------+-----+
|Avec réduction| 2518|
|Sans réduction|53173|
+--------------+-----+



Il y a une majorité de jeux sans réduction sur Steam, mais voyons la même chose en %

In [0]:
# D'abord, calculons le total des jeux
total_games = df.count()

# Maintenant faisons la même requête avec un pourcentage
df.select('data.discount')\
  .withColumn('has_discount',
    F.when(F.col('discount') != '0', 'Avec réduction')
    .otherwise('Sans réduction'))\
  .groupBy('has_discount')\
  .count()\
  .withColumn('percentage', 
    F.round(F.col('count') * 100 / total_games, 2))\
  .show()

+--------------+-----+----------+
|  has_discount|count|percentage|
+--------------+-----+----------+
|Avec réduction| 2518|      4.52|
|Sans réduction|53173|     95.48|
+--------------+-----+----------+



4,45 % des jeux ont été en promotion sur steam. Affinons avec le pourcentage moyen de réduction

In [0]:
# pourcentage moyen de réduction
df.select('data.discount')\
  .where(F.col('discount') != '0')\
  .select(F.avg(F.col('discount').cast('float')))\
  .show()

+----------------------------+
|avg(CAST(discount AS FLOAT))|
+----------------------------+
|           57.58816521048451|
+----------------------------+



La moyenne est subsentiel : 57.59 % ! Finissons par classer cela dans des catégories pour mieux visualiser les politiques de promotions

In [0]:
#classification des promotions
df.select('data.discount')\
  .withColumn('discount_num', F.col('discount').cast('float'))\
  .withColumn('discount_category',
    F.when(F.col('discount_num') == 0, 'Pas de réduction')
    .when(F.col('discount_num') <= 25, 'Petite réduction (1-25%)')
    .when(F.col('discount_num') <= 50, 'Réduction moyenne (26-50%)')
    .when(F.col('discount_num') <= 75, 'Grande réduction (51-75%)')
    .otherwise('Très grande réduction (76-100%)'))\
  .groupBy('discount_category')\
  .count()\
  .orderBy('count', ascending=False)\
  .show(truncate=False)

+-------------------------------+-----+
|discount_category              |count|
+-------------------------------+-----+
|Pas de réduction               |53173|
|Grande réduction (51-75%)      |815  |
|Réduction moyenne (26-50%)     |771  |
|Très grande réduction (76-100%)|623  |
|Petite réduction (1-25%)       |309  |
+-------------------------------+-----+




## What are the most represented languages?

In [0]:
# contenu de la colonne "data.languages"
df.select('data.languages')\
  .limit(5)\
  .show(truncate=False)

+-------------------------------------------------------------------------------------------------------------------------+
|languages                                                                                                                |
+-------------------------------------------------------------------------------------------------------------------------+
|English, French, German, Italian, Spanish - Spain, Simplified Chinese, Traditional Chinese, Korean                       |
|English, Korean, Simplified Chinese                                                                                      |
|Simplified Chinese, English, Japanese, Traditional Chinese, French, German, Spanish - Spain, Russian, Portuguese - Brazil|
|English                                                                                                                  |
|Simplified Chinese                                                                                                       |
+-------

Le problème ici est que chaque jeux peut être en plusieurs langues.La prochaine requête devra : prendre la colonne "languages", la diviser aux virgules, créer une ligne par langue (explode), nettoyer les espaces (trim), grouper, compter et enfin ordonner par fréquence

In [0]:
# classement des langues les plus utilisés
df.select('data.languages')\
  .select(F.explode(F.split('languages', ',')))\
  .select(F.trim(F.col('col')).alias('language'))\
  .groupBy('language')\
  .count()\
  .orderBy('count', ascending=False)\
  .show()

+--------------------+-----+
|            language|count|
+--------------------+-----+
|             English|55116|
|              German|14019|
|              French|13426|
|             Russian|12922|
|  Simplified Chinese|12782|
|     Spanish - Spain|12233|
|            Japanese|10368|
|             Italian| 9304|
| Portuguese - Brazil| 6750|
|              Korean| 6600|
| Traditional Chinese| 6307|
|              Polish| 5376|
|Portuguese - Port...| 4011|
|             Turkish| 3602|
|               Dutch| 3080|
|Spanish - Latin A...| 2732|
|               Czech| 2352|
|             Swedish| 2047|
|           Ukrainian| 1928|
|           Hungarian| 1923|
+--------------------+-----+
only showing top 20 rows



## Are there many games prohibited for children under 16/18?

Encore une formulation un peu ambigue. Nous allons calculons le nombre de jeux interdits aux moins de 16 ans et au moins de 18 ans

In [0]:
# contenu de la colonne "data.required_age"
df.select('data.required_age')\
  .groupBy('required_age')\
  .count()\
  .orderBy('required_age')\
  .show()

+------------+-----+
|required_age|count|
+------------+-----+
|           0|55030|
|          10|    7|
|          12|   32|
|          13|   26|
|          14|   10|
|          15|  264|
|          16|   38|
|          17|   38|
|          18|  223|
|         180|    4|
|          20|    1|
|         21+|    1|
|           3|    3|
|          35|    1|
|           5|    1|
|           6|    4|
|           7|    2|
|          7+|    1|
|           8|    3|
|           9|    1|
+------------+-----+
only showing top 20 rows



On constate quelques erreurs ou manières de notation peu classique : 
- 180 est surement pour 18.
- Considérons également les chiffres 21+ et 35 comme plus de 18 ans

In [0]:
# nombre de jeux interdits aux moins de 16 ans avec pourcentage
df.select('data.required_age')\
  .where(F.col('required_age').isin(['16', '17', '18', '180',20, '21+', '35']))\
  .agg(
    F.count('*').alias('jeux_min16'),
    (F.count('*') * 100 / df.count()).alias('pourcentage_min16')
  )\
  .show()

+----------+-----------------+
|jeux_min16|pourcentage_min16|
+----------+-----------------+
|       306|0.549460415506994|
+----------+-----------------+



In [0]:
#nombre de jeux interdits aux moins de 18 ans avec pourcentage
df.select('data.required_age')\
  .where(F.col('required_age').isin(['18', '180', 20, '21+', '35']))\
  .agg(
    F.count('*').alias('jeux_min18'),
    (F.count('*') * 100 / df.count()).alias('pourcentage_min18')
  )\
  .show()

+----------+------------------+
|jeux_min18| pourcentage_min18|
+----------+------------------+
|       230|0.4129931227666948|
+----------+------------------+



Il y a peu de jeux avec une limite d'âge haute

## Are most games available on Windows/Mac/Linux?

Attention dans ces colonnes, il s'agit de booleans. Nous utiliserons .cast('int') afin de convertir True en 1 et False en 0

In [0]:
# nombre de jeux sous window/mac/linux en porucentage
df.select(
    (F.avg(F.col('data.platforms.windows').cast('int')) * 100).alias('windows_%'),
    (F.avg(F.col('data.platforms.mac').cast('int')) * 100).alias('mac_%'),
    (F.avg(F.col('data.platforms.linux').cast('int')) * 100).alias('linux_%')
)\
.show()

+-----------------+------------------+------------------+
|        windows_%|             mac_%|           linux_%|
+-----------------+------------------+------------------+
|99.97306566590652|22.930096424916055|15.187373184176977|
+-----------------+------------------+------------------+



## What are the most represented genres?

Comme précédement avec la langue, on constate que certains jeux ont plusieurs style. On adapte donc la requête comme avec la lanque précédement

In [0]:
# Style de jeux le plus représenté :
df.select('data.genre')\
  .select(F.explode(F.split('genre', ',')))\
  .select(F.trim(F.col('col')).alias('genre'))\
  .groupBy('genre')\
  .count()\
  .orderBy('count', ascending=False)\
  .show()

+--------------------+-----+
|               genre|count|
+--------------------+-----+
|               Indie|39681|
|              Action|23759|
|              Casual|22086|
|           Adventure|21431|
|            Strategy|10895|
|          Simulation|10836|
|                 RPG| 9534|
|        Early Access| 6145|
|        Free to Play| 3393|
|              Sports| 2666|
|              Racing| 2155|
|Massively Multipl...| 1460|
|           Utilities|  682|
|Design & Illustra...|  406|
|Animation & Modeling|  322|
|           Education|  317|
|    Video Production|  247|
|    Audio Production|  195|
|             Violent|  168|
|   Software Training|  164|
+--------------------+-----+
only showing top 20 rows



Nous avons identifié le style de jeu le plus présent sur Stream : le jeu indépendant. Les jeux d'actions et casual "grand pulic" sont également dand le top 3.
Mais les jeux indépendants sont-ils ceux que les joueurs préfèrent ?

In [0]:
# Style de jeux préféré selon le nombre d'avis positif total
df.select(
    F.explode(F.split('data.genre', ',')).alias('genre'),
    'data.positive'
)\
.select(
    F.trim(F.col('genre')).alias('genre'),
    'positive'
)\
.groupBy('genre')\
.agg(F.sum('positive').alias('total_avis_positifs'))\
.orderBy('total_avis_positifs', ascending=False)\
.show()

+--------------------+-------------------+
|               genre|total_avis_positifs|
+--------------------+-------------------+
|              Action|           54858618|
|               Indie|           32531023|
|           Adventure|           29689445|
|                 RPG|           19425528|
|        Free to Play|           18722246|
|          Simulation|           15572390|
|            Strategy|           13402870|
|              Casual|           10034967|
|Massively Multipl...|            7979078|
|        Early Access|            4334595|
|              Sports|            2949805|
|              Racing|            2340353|
|           Utilities|             739335|
|Animation & Modeling|             690765|
|Design & Illustra...|             674057|
|       Photo Editing|             577751|
|                    |             189649|
|    Video Production|             111514|
|    Audio Production|              69118|
|             Violent|              41140|
+----------

Pas tout à fait !

## Question supplémentaires en vu d'améliorer les préconisations à Ubisoft

Pour répondre un peu mieux à la demande, nous rajoutons quelques questions.

Par exemple quel est le chiffre d'affaire total par rapport au genre (par exemple prix jeux d'aventure * le nombre de propriétaire). 
- depuis la création de STEAM
- depuis les 3 dernières années pour voir la tendance actuelle

In [0]:
# contenu des colonnes price et owners
df.select('data.price', 'data.owners', 'data.genre').show(5)

+-----+--------------------+--------------------+
|price|              owners|               genre|
+-----+--------------------+--------------------+
|  999|10,000,000 .. 20,...|              Action|
|  999|         0 .. 20,000|Action, Adventure...|
|  599|  200,000 .. 500,000|Adventure, Indie,...|
| 1999|  100,000 .. 200,000|Action, Indie, Si...|
|  199|         0 .. 20,000|Action, Casual, I...|
+-----+--------------------+--------------------+
only showing top 5 rows



Steam fourni une plage pour le nombre de propriétaire. La ligne 1 indique qu'il y a eu entre 10 000 000 et 20 000 000 acheteurs d'un jeux d'action coutant 9.99 dollars. Pour la suite nous prendrons la plage minimale d'acheteur. Plus prudent.

In [0]:
# chiffre d'affaires moyen par jeu dans chaque genre 
df.select(
    F.explode(F.split('data.genre', ',')).alias('genre'),
    F.col('data.price').cast('float').alias('price'),
    F.regexp_replace(F.split('data.owners', ' \\.\\. ')[0], ',', '').cast('float').alias('min_owners')
)\
.groupBy('genre')\
.agg(
    F.round(F.sum(F.col('price') * F.col('min_owners'))/1000000000, 2).alias('CA_total_milliards$'),
    F.count('*').alias('nombre_jeux'),
    F.round(F.sum(F.col('price') * F.col('min_owners')) / F.count('*')/1000000, 2).alias('CA_moyen_par_jeu_millions$')
)\
.orderBy('CA_moyen_par_jeu_millions$', ascending=False)\
.show()

+--------------------+-------------------+-----------+--------------------------+
|               genre|CA_total_milliards$|nombre_jeux|CA_moyen_par_jeu_millions$|
+--------------------+-------------------+-----------+--------------------------+
|                 RPG|             309.38|        815|                    379.61|
|            Strategy|             275.12|        850|                    323.67|
| Massively Multip...|              363.0|       1412|                    257.08|
|          Simulation|              250.0|       1207|                    207.12|
| Animation & Mode...|               8.21|         42|                    195.47|
|              Racing|               66.1|        357|                    185.14|
|           Adventure|            1727.25|      10166|                    169.91|
|                 RPG|            1358.13|       8719|                    155.77|
|              Action|            3591.63|      23616|                    152.08|
|Massively Multi

Regardons au cours le comportement au cours des 2 dernières années. Exluons 2020 qui est l'année COVID et avec beaucoup de jeux sortis, peut-être un comportement atypique avec le confinement...un peu biaisé...

In [0]:
# chiffre d'affaires moyen par jeu dans chaque genre entre 2021 et 2022 (2 dernières années)
df.select(
    F.explode(F.split('data.genre', ',')).alias('genre'),
    F.col('data.price').cast('float').alias('price'),
    F.regexp_replace(F.split('data.owners', ' \\.\\. ')[0], ',', '').cast('float').alias('min_owners'),
    'data.release_date'
)\
.where(F.substring('release_date', 1, 4).isin(['2021', '2022']))\
.groupBy('genre')\
.agg(
    F.round(F.sum(F.col('price') * F.col('min_owners'))/1000000000, 2).alias('CA_total_milliards$'),
    F.count('*').alias('nombre_jeux'),
    F.round(F.sum(F.col('price') * F.col('min_owners')) / F.count('*')/1000000, 2).alias('CA_moyen_par_jeu_millions$')
)\
.orderBy('CA_moyen_par_jeu_millions$', ascending=False)\
.show()

+--------------------+-------------------+-----------+--------------------------+
|               genre|CA_total_milliards$|nombre_jeux|CA_moyen_par_jeu_millions$|
+--------------------+-------------------+-----------+--------------------------+
| Massively Multip...|             128.41|        339|                    378.78|
|              Racing|               21.6|         93|                     232.3|
|                 RPG|             475.93|       2796|                    170.22|
|          Simulation|              55.77|        354|                    157.54|
|           Adventure|             380.99|       3379|                    112.75|
|              Action|             709.63|       6867|                    103.34|
|        Early Access|               0.17|          2|                     84.95|
|              Sports|               56.5|        692|                     81.65|
|          Simulation|             190.92|       2966|                     64.37|
|            Str

In [0]:
# A quel moment sort les jeux ?
df.select(
    F.substring('data.release_date', 6, 2).alias('month'))\
.groupBy('month')\
.count()\
.orderBy('month')\
.show()

+-----+-----+
|month|count|
+-----+-----+
|     |   99|
|   01| 3877|
|   02| 4219|
|   03| 4708|
|   04| 4470|
|   05| 4731|
|   06| 4212|
|   07| 4767|
|   08| 5077|
|   09| 5177|
|   10| 5549|
|   11| 4557|
|   12| 4248|
+-----+-----+



Terminons par deux onformations commerciales : 
- à quelle moment faut-il mieux sortir un jeu ?
- on a identifié un prix potentiel sur toute la bie de steam, mais n'y a-t-il pas une évolution des prix ces dernières années ?

In [0]:
# mois des sorties ( 6, 2 car forme "2021/05/14" dans la colonne )
df.select(
    F.substring('data.release_date', 6, 2).alias('month')
)\
.groupBy('month')\
.count()\
.orderBy('month')\
.show()

+-----+-----+
|month|count|
+-----+-----+
|     |   99|
|   01| 3877|
|   02| 4219|
|   03| 4708|
|   04| 4470|
|   05| 4731|
|   06| 4212|
|   07| 4767|
|   08| 5077|
|   09| 5177|
|   10| 5549|
|   11| 4557|
|   12| 4248|
+-----+-----+



Saisonnalité des sorties :

- pic 1 est en septembre-octobre (5177 et 5549 jeux)
- ic 2 en juillet-août (4767 et 5077 jeux)
- Creux en janvier-février (3877 et 4219 jeux)

Ce qui montre des moments stratégiques de sortie : rentrée scolaire et vacances d'été

In [0]:
# L'évolution des prix pour savoir à quel tarif vendre
df.select(
    F.substring('data.release_date', 1, 4).alias('year'),
    F.col('data.price').cast('float').alias('price')
)\
.groupBy('year')\
.agg(F.avg('price')/100).alias('prix_moyen')\
.orderBy('year')\
.show(30)

+----+------------------+
|year|(avg(price) / 100)|
+----+------------------+
|    | 4.620707070707071|
|1997|             14.98|
|1998|              9.99|
|1999|3.3266666666666667|
|2000|              7.49|
|2001|              9.99|
|2002|             14.99|
|2003| 8.906666666666666|
|2004|              9.99|
|2005| 7.991666666666666|
|2006| 9.024426229508197|
|2007| 6.721530612244898|
|2008|  8.53006289308176|
|2009| 9.216334405144694|
|2010|         7.9984375|
|2011| 9.067902621722846|
|2012| 9.573304347826086|
|2013|11.020806794055202|
|2014| 9.012710340398202|
|2015| 7.533373447204968|
|2016| 7.214410991636798|
|2017| 6.825487784610271|
|2018| 6.308491794738213|
|2019| 7.261186854190586|
|2020| 7.226219145093317|
|2021|  8.51796214439533|
|2022| 9.709977196512408|
+----+------------------+



- Une relative stabilité des prix moyens entre 7$ et 11$ sur la période (juste une seule année au delà à 15$).
- Une égère tendance à la hausse jusqu'en 2013 (pic à 11.02$) puis baisse progressive (7.53$ en 2015)
- Attention à ne pas vendre trop cher, les prix n'augmentent pas forcément avec le temps !

## CONCLUSION


## ANALYSE DU MARCHÉ DES JEUX VIDÉO SUR STEAM

## 1. Tendances du Marché
- **Croissance** : De 471 jeux (2013) à 8823 jeux (2021, pic historique)
- **Saisonnalité** : 
  * Pics : Septembre-Octobre et Juillet-Août
  * Creux : Janvier-Février
- **Distribution par plateforme** :
  * Windows : 99.97%
  * Mac : 22.93%
  * Linux : 15.19%

## 2. Analyse Économique
- **Structure des prix** :
  * Prix moyen : 7.73$ (stable entre 7-11$)
  * < 5$ : 42.16% des jeux
  * 5-9.99$ : 22.35%
  * Gratuits : 13.97%
- **Politique de remises** :
  * 4.52% des jeux en promotion
  * Remise moyenne : 57.59%

## 3. Analyse des Genres
- **Plus représentés** :
  * Indie
  * Action
  * Casual
- **Avis positifs** :
  * Action
  * Indie
  * Adventure
- **Plus rentables** (CA moyen par jeu) :
  * RPG (379.61M$)
  * Strategy (323.67M$)
  * Massively Multiplayer (257.08M$)

## 4. Localisation et Accessibilité
- **Distribution linguistique** :
  * Anglais : 98.9%
  * Allemand : 25.2%
  * Français : 24.1%
  * Russe : 23.2%
  * Chinois simplifié : 22.9%
- **Restrictions d'âge** :
  * 16+ : 0.55%
  * 18+ : 0.41%

## RECOMMANDATIONS POUR UBISOFT

## 1. Stratégie de Développement
- Focus prioritaire sur Windows
- Minimum 5 langues : EN, DE, FR, RU, CN
- Contenu tout public (très peu de jeux 18+)

## 2. Genre et Positionnement
- Privilégier RPG ou Massively Multiplayer (rentabilité)
- Alternative : Action (volume important)
- Possibilité d'hybridation avec Adventure

## 3. Stratégie Commerciale
- **Timing** : Sortie en Septembre-Octobre (ou Juillet-Août)
- **Prix** : 
  * Lancement : 7-11$
  * Remises prévues : 50-60%

Cette stratégie permettra de :
- Maximiser le potentiel commercial
- Optimiser le timing de sortie
- Atteindre une audience large
- Se positionner sur des segments rentables